# RGB Baseline Models (EuroSAT)

This notebook trains baseline RGB image classification models on the EuroSAT dataset using the fixed train/validation/test splits defined in `01_dataset_preparation_and_splits.ipynb`

The goal is to establish a clean performance baseline before introducing multispectral inputs or physics-aware inductive biases.

---

In [1]:
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, models
from torchvision.datasets import EuroSAT

# Project root (robust to running from notebooks/)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

DATA_DIR = PROJECT_ROOT / "data"
SPLITS_DIR = PROJECT_ROOT / "splits"

In [2]:
splits = np.load(SPLITS_DIR / "eurosat_rgb_splits.npz")

train_idx = splits["train_idx"]
val_idx   = splits["val_idx"]
test_idx  = splits["test_idx"]

print(len(train_idx), len(val_idx), len(test_idx))

21600 2700 2700


In [3]:
rgb_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

ds_rgb = EuroSAT(
    root=str(DATA_DIR / "eurosat_rgb"),
    download=False,   # already downloaded
    transform=rgb_transform
)

train_ds = Subset(ds_rgb, train_idx)
val_ds   = Subset(ds_rgb, val_idx)
test_ds  = Subset(ds_rgb, test_idx)

In [4]:
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=128, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=128, shuffle=False, num_workers=0)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(ds_rgb.classes)

model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
def train_one_epoch(model, loader):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)

    return total_loss / total, correct / total

In [9]:
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)

    return correct / total

In [10]:
epochs = 5

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader)
    val_acc = evaluate(model, val_loader)

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Train loss: {train_loss:.4f} | "
        f"Train acc: {train_acc:.4f} | "
        f"Val acc: {val_acc:.4f}"
    )

Epoch 1/5 | Train loss: 0.8944 | Train acc: 0.6863 | Val acc: 0.4352
Epoch 2/5 | Train loss: 0.6360 | Train acc: 0.7769 | Val acc: 0.6641
Epoch 3/5 | Train loss: 0.4899 | Train acc: 0.8296 | Val acc: 0.6715
Epoch 4/5 | Train loss: 0.4149 | Train acc: 0.8563 | Val acc: 0.6715
Epoch 5/5 | Train loss: 0.3613 | Train acc: 0.8750 | Val acc: 0.7274


In [11]:
test_acc = evaluate(model, test_loader)
print("Test accuracy:", test_acc)

Test accuracy: 0.7344444444444445
